#Собрать информацию о вакансиях на вводимую должность с сайтов hh.ru и/или Superjob и/или работа.ру. Приложение должно анализировать несколько страниц сайта. Получившийся список должен содержать в себе минимум:
Наименование вакансии.
Предлагаемую зарплату (дополнительно: разносим в три поля: минимальная и максимальная и валюта. цифры преобразуем к цифрам).
Ссылку на саму вакансию.
Сайт, откуда собрана вакансия.
По желанию можно добавить ещё параметры вакансии (например, работодателя и расположение). Структура должна быть одинаковая для вакансий с всех сайтов. Общий результат можно вывести с помощью dataFrame через pandas, сохранить в json, либо csv.

In [28]:
from bs4 import BeautifulSoup as bs
import requests
from time import sleep


In [29]:
num_page = 0
url = 'https://hh.ru/vacancies/analitik-dannyh?page=num_page'
headers = {
    'User-Agent':	'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36',
}

list_dom = []
for page in range(2):
  response = requests.get(url = url, headers = headers)
  num_page +=1
  print(response.status_code)
  dom = bs(response.text, 'html.parser')
  tag_vacancy = dom.find_all('div', class_= 'serp-item')
  print(len(tag_vacancy))
  list_dom.append(tag_vacancy)
  sleep(3)

200
50
200
50


In [ ]:
number = 1
dict_vacancy = {}
site_name = 'hh.ru'
for dom in list_dom:
  for vacancy in tag_vacancy:
    vacancy_title = vacancy.find('a', attrs={'class': 'bloko-link', 'data-qa': 'vacancy-serp__vacancy-title'}).string
    if vacancy.find('span', attrs={'class': 'bloko-header-section-3', 'data-qa': 'vacancy-serp__vacancy-compensation'}) == None:
      min_compensation = None
      max_compensation = None
      currency = None
    else:
      compensation = vacancy.find('span', attrs={'class': 'bloko-header-section-3', 'data-qa': 'vacancy-serp__vacancy-compensation'}).text.replace(' ', '').split()
      if compensation[0] == 'от':
        min_compensation = int(compensation[1])
        max_compensation = None
        currency = compensation[2]
      elif compensation[0] == 'до':
        min_compensation = None
        max_compensation = int(compensation[1])
        currency = compensation[2]
      else:
        min_compensation = int(compensation[0])
        max_compensation = int(compensation[2])
        currency = compensation[3]
    vacancy_link =  vacancy.find('a', attrs={'class': 'bloko-link', 'data-qa': 'vacancy-serp__vacancy-title'}).get('href')
    dict_vacancy[number] = {
         'должность': vacancy_title,
         'минимальная зарплата' : min_compensation,
         'максимальная зарплата' : max_compensation,
         'валюта' : currency,
         'ссылка на вакансию' : vacancy_link,
         'сайт' : site_name
         }
    number +=1

  
for key in dict_vacancy.keys():
   print(f'{key} - {dict_vacancy[key]}')

In [31]:
import pandas as pd

In [32]:
vacancy_dataframe = pd.DataFrame(dict_vacancy).T
vacancy_dataframe


,должность,минимальная зарплата,максимальная зарплата,валюта,ссылка на вакансию,сайт
1,Аналитик данных (Senior),None,None,None,https://hh.ru/vacancy/69539016?query=%D0%B0%D0...,hh.ru
2,Диспонент / Аналитик данных со знанием английс...,100000,None,руб.,https://hh.ru/vacancy/68779294?query=%D0%B0%D0...,hh.ru
3,Аналитик BI,None,None,None,https://hh.ru/vacancy/69589522?query=%D0%B0%D0...,hh.ru
4,Директор (аналитик данных) Отдела портфельного...,None,None,None,https://hh.ru/vacancy/68076483?query=%D0%B0%D0...,hh.ru
5,Букмекер-аналитик,None,None,None,https://hh.ru/vacancy/54499134?query=%D0%B0%D0...,hh.ru
...,...,...,...,...,...,...
96,Бизнес-аналитик (стажёр),25000,None,руб.,https://hh.ru/vacancy/69541774?query=%D0%B0%D0...,hh.ru
97,Аналитик данных (data analyst),None,120000,руб.,https://hh.ru/vacancy/69634789?query=%D0%B0%D0...,hh.ru
98,Аналитик BI,None,None,None,https://hh.ru/vacancy/69389745?query=%D0%B0%D0...,hh.ru
99,Аналитик данных в команду управления эффективн...,None,None,None,https://hh.ru/vacancy/69634264?query=%D0%B0%D0...,hh.ru


In [33]:
import json
with open('dict_vacancy', "w", encoding='utf-8') as json_file:
    json.dump(dict_vacancy, json_file)
    print(f'файл создан успешно, в списке - {number - 1} вакансий')

файл создан успешно, в списке - 100 вакансий
